In [48]:
import pandas as pd
import os
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import sklearn.metrics
from collections import defaultdict
import shutil

pd.reset_option('all')

As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.

: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



In [49]:
table = '/Users/rijulvohra/Documents/work/Novartis-ISI/table_linker_pipeline/'
table_name = 'players.csv'
wikify_column_name = "Player (2011 TBs)"

In [50]:
#intermediate files
canonical = 'canonical.csv'
candidates = "candidates.csv"
feature_votes = "feature_votes.csv"
score_file = "scores.csv"
temp_dir = os.path.join(os.path.join(table,'temp'))
table_path = os.path.join(table,table_name)
if not(os.path.exists(temp_dir)):
    os.makedirs(temp_dir)

## Canonicalize

In [51]:
canonical_file = os.path.join(temp_dir,canonical)
!tl canonicalize -c "$wikify_column_name" --add-context $table_path > $canonical_file

In [52]:
pd.read_csv(canonical_file, nrows = 10)

,column,row,label,context
0,1,0,Hank Aaron,"1|6,856"
1,1,1,Stan Musial,"2|6,134"
2,1,2,Willie Mays,"3|6,066"
3,1,3,Barry Bonds,"4|5,976"
4,1,4,Ty Cobb,"5|5,854"
5,1,5,Babe Ruth,"6|5,793"
6,1,6,Pete Rose,"7|5,752"
7,1,7,Carl Yastrzemski,"8|5,539"
8,1,8,Eddie Murray,"9|5,397"
9,1,9,Rafael Palmeiro,"10|5,388"


## Candidate Generation

In [53]:
candidates_file = os.path.join(temp_dir,candidates)
!tl clean -c label -o label_clean $canonical_file / get-fuzzy-augmented-matches --es-url http://ckg06:9200 \
                                                    --es-index wikidata_augmented_index -c label_clean \
            / --url http://ckg06:9200 --index wikidataos-07 get-exact-matches -c label_clean > $candidates_file

In [54]:
pd.read_csv(candidates_file,nrows = 10)

,column,row,label,context,label_clean,kg_id,kg_labels,method,retrieval_score,pagerank,kg_descriptions
0,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q47513596,Hank Aaron,fuzzy-augmented,17.210197,6.993175e-09,painting by Ross R. Rossin
1,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q215777,Hank Aaron,fuzzy-augmented,37.566580,1.859623e-08,American baseball player
2,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q14679126,Hank Aaron Stadium,fuzzy-augmented,28.132477,4.207063e-08,NaN
3,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q1518478,Hank Aaron Award,fuzzy-augmented,19.771885,8.273086e-09,annual baseball award given to the top hitter ...
4,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q92433937,Reflections on Hank Aaron,fuzzy-augmented,26.248425,0.000000e+00,scientific article published on 29 April 2019
5,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q28453830,Hank Aaron State Trail,fuzzy-augmented,26.248425,0.000000e+00,NaN
6,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q6665277,Template:AL Hank Aaron Award Winners,fuzzy-augmented,24.600891,0.000000e+00,Wikimedia template
7,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q8853836,Template:NL Hank Aaron Award Winners,fuzzy-augmented,24.600891,0.000000e+00,Wikimedia template
8,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q5648263,Hank Aaron: Chasing the Dream,fuzzy-augmented,24.600891,6.993175e-09,1995 film by Michael Tollin
9,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q66847614,President Carter with Hank Aaron (NAID 180805),fuzzy-augmented,21.857079,6.993175e-09,file unit in the National Archives and Records...


## Feature Voting

In [55]:
%%time
feature_votes_file = os.path.join(temp_dir,feature_votes)
!tl smallest-qnode-number $candidates_file \
    / string-similarity -i --method monge_elkan:tokenizer=word -o monge_elkan \
    / string-similarity -i --method jaccard:tokenizer=word -c kg_descriptions context -o des_cont_jaccard \
    / feature-voting -c "pagerank,smallest_qnode_number,monge_elkan,des_cont_jaccard" > $feature_votes_file

CPU times: user 77.6 ms, sys: 33.2 ms, total: 111 ms
Wall time: 5.21 s


In [56]:
pd.read_csv(feature_votes_file,nrows = 10)

,column,row,label,context,label_clean,kg_id,kg_labels,method,retrieval_score,pagerank,kg_descriptions,smallest_qnode_number,monge_elkan,des_cont_jaccard,votes
0,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q47513596,Hank Aaron,fuzzy-augmented,17.210197,6.993175e-09,painting by Ross R. Rossin,0,1.000000,0,1
1,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q215777,Hank Aaron,fuzzy-augmented,37.566580,1.859623e-08,American baseball player,0,1.000000,0,1
2,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q14679126,Hank Aaron Stadium,fuzzy-augmented,28.132477,4.207063e-08,NaN,0,1.000000,0,1
3,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q1518478,Hank Aaron Award,fuzzy-augmented,19.771885,8.273086e-09,annual baseball award given to the top hitter ...,0,1.000000,0,1
4,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q92433937,Reflections on Hank Aaron,fuzzy-augmented,26.248425,0.000000e+00,scientific article published on 29 April 2019,0,1.000000,0,1
5,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q28453830,Hank Aaron State Trail,fuzzy-augmented,26.248425,0.000000e+00,NaN,0,1.000000,0,1
6,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q6665277,Template:AL Hank Aaron Award Winners,fuzzy-augmented,24.600891,0.000000e+00,Wikimedia template,0,1.000000,0,1
7,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q8853836,Template:NL Hank Aaron Award Winners,fuzzy-augmented,24.600891,0.000000e+00,Wikimedia template,0,1.000000,0,1
8,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q5648263,Hank Aaron: Chasing the Dream,fuzzy-augmented,24.600891,6.993175e-09,1995 film by Michael Tollin,0,0.983333,0,0
9,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q66847614,President Carter with Hank Aaron (NAID 180805),fuzzy-augmented,21.857079,6.993175e-09,file unit in the National Archives and Records...,0,1.000000,0,1


## Compute Embedding Score using Column Vector Strategy

In [57]:
%%time
score = os.path.join(temp_dir,score_file)
!tl score-using-embedding $feature_votes_file --column-vector-strategy centroid-of-voting -o graph-embedding-score \
--min-vote 2 --embedding-file /tmp/wikidataos-graph-embedding-01.tsv \
--embedding-url http://kg2018a.isi.edu:9200/wikidataos-graph-embedding-01/doc/ \
> $score

Qnodes to lookup: 3042
Qnodes from file: 2913
Qnodes from server: 0
CPU times: user 28.4 ms, sys: 19.1 ms, total: 47.5 ms
Wall time: 1.66 s


In [58]:
pd.read_csv(score,nrows = 10)

,column,row,label,context,label_clean,kg_id,kg_labels,method,retrieval_score,pagerank,kg_descriptions,smallest_qnode_number,monge_elkan,des_cont_jaccard,votes,graph-embedding-score
0,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q47513596,Hank Aaron,fuzzy-augmented,17.210197,6.993175e-09,painting by Ross R. Rossin,0,1.000000,0,1,0.292322
1,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q215777,Hank Aaron,fuzzy-augmented,37.566580,1.859623e-08,American baseball player,0,1.000000,0,1,0.281958
2,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q14679126,Hank Aaron Stadium,fuzzy-augmented,28.132477,4.207063e-08,NaN,0,1.000000,0,1,0.318894
3,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q1518478,Hank Aaron Award,fuzzy-augmented,19.771885,8.273086e-09,annual baseball award given to the top hitter ...,0,1.000000,0,1,0.264204
4,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q92433937,Reflections on Hank Aaron,fuzzy-augmented,26.248425,0.000000e+00,scientific article published on 29 April 2019,0,1.000000,0,1,NaN
5,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q28453830,Hank Aaron State Trail,fuzzy-augmented,26.248425,0.000000e+00,NaN,0,1.000000,0,1,NaN
6,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q6665277,Template:AL Hank Aaron Award Winners,fuzzy-augmented,24.600891,0.000000e+00,Wikimedia template,0,1.000000,0,1,0.394617
7,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q8853836,Template:NL Hank Aaron Award Winners,fuzzy-augmented,24.600891,0.000000e+00,Wikimedia template,0,1.000000,0,1,0.383239
8,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q5648263,Hank Aaron: Chasing the Dream,fuzzy-augmented,24.600891,6.993175e-09,1995 film by Michael Tollin,0,0.983333,0,0,0.253758
9,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q66847614,President Carter with Hank Aaron (NAID 180805),fuzzy-augmented,21.857079,6.993175e-09,file unit in the National Archives and Records...,0,1.000000,0,1,0.314680


## Compute final score

In [59]:
df = pd.read_csv(score)

In [60]:
def zscore_normalization(df,column):
    normalized_column = 'normalized-' + column
    df[normalized_column] = (df[column] - df[column].mean())/df[column].std() 
    return df
df = zscore_normalization(df,'graph-embedding-score')
df = zscore_normalization(df,'pagerank')
df = zscore_normalization(df,'monge_elkan')

In [61]:
def calc_sum(x):
    graph_score = x['normalized-graph-embedding-score']
    pagerank = x['normalized-pagerank']
    monge_elkan = x['normalized-monge_elkan']
    l = [graph_score,pagerank,monge_elkan]
    l1 = list(filter(lambda x: not(pd.isna(x)),l))
    ans = sum(l1)
    return ans
df['final_score'] = df.apply(lambda x: calc_sum(x),axis = 1)

## Get Top5 KG Links

In [62]:
top5_df = df.groupby(['column','row']).apply(lambda x: x.sort_values(['final_score'],ascending=False)[:5]).reset_index(drop = True)

In [63]:
def get_kg_links(top5_df,input_df):
    k = defaultdict(list)
    for i, row in top5_df.iterrows():
        col_row_key = str(row['column']) + '_' + str(row['row'])
        k[col_row_key].append((row['kg_id'],row['kg_labels'],row['final_score']))
        
    output_df_list = []
    for key in k:
        col,row = int(key.split('_')[0]),int(key.split('_')[1])
        wikify_col = input_df.columns[col]
        kg_id = wikify_col + '_kg_id'
        kg_label = wikify_col + '_kg_label'
        kg_score = wikify_col + '_score'
        for val in k[key]:
            _ = {}
            for c in input_df.columns:
                _[c] = input_df.loc[row,c]
            _[kg_id] = val[0]
            _[kg_label] = val[1]
            _[kg_score] = val[2]
            output_df_list.append(_)
    output_df = pd.DataFrame(output_df_list)
    
    return output_df

input_df = pd.read_csv(table_path)
output_df = get_kg_links(top5_df, input_df)

In [64]:
output_df.head(10)

,Rank,Player (2011 TBs),TBs,Player (2011 TBs)_kg_id,Player (2011 TBs)_kg_label,Player (2011 TBs)_score
0,1,Hank Aaron,"6,856",Q64138596,Hank Hartman,2.434898
1,1,Hank Aaron,"6,856",Q95732001,Dagmar Hank,2.285370
2,1,Hank Aaron,"6,856",Q90169741,Jacquelyn A Hank,2.275743
3,1,Hank Aaron,"6,856",Q534842,Hank Mobley,2.225323
4,1,Hank Aaron,"6,856",Q89188155,Hank Edelhauser,2.034626
5,2,Stan Musial,"6,134",Q1453,Alexandra Stan,5.362707
6,2,Stan Musial,"6,134",Q90060810,George Stan,3.208164
7,2,Stan Musial,"6,134",Q12739615,Roxana Stan,3.116906
8,2,Stan Musial,"6,134",Q57270246,Camilla Stan,3.100711
9,2,Stan Musial,"6,134",Q58240717,Laura Stan,2.877054


In [65]:
linked_table_name = "linked-" + table_name
output_path = os.path.join(table,linked_table_name)
output_df.to_csv(output_path,index = False)

## CleanUp temporary files

In [66]:
shutil.rmtree(temp_dir)